# Logisztikus regresszió alkalmazása multiklasszifkációs problémára

## Feladat rövid leírása

A feladat során az xy multiklasszifkációs adatbázis felhasználásával mutatom be a logisztikus regresszió alkalmazását amely segitségével a megfelelő adatisztási feladatok elvégzésvel n.k%-os pontosságot sikerült elérnem.

## Készítette:
### Név: Gipsz Jakab
### Neptun kód: GIP5JAK

## Adatok előkészítése

### Nyers adat beolvasása

In [ ]:
import pandas as pd

df_news = pd.read_json("data/news.json", orient='records')
df_news.head()

### A szükségtelen jellemzők eltávolítása

In [ ]:
df_news_filtered = pd.DataFrame(columns=["text","label","label_name"])
df_news_filtered["text"] = [item for item in df_news["content"].values]
df_news_filtered["label_name"] = [item["label"][0] for item in df_news["annotation"].values]
df_news_filtered.head()

### Hisztogram az adatokban lévő osztályok eloszlásáról

A hisztogramon jól látható hogy az egyes csoportok egyenlő számban tartalmznak elemeket az egyes osztályokhoz. Ezért kijelenthető hogy az adatok az osztályok szempontjából egyenelet eloszlásuak. Így az adathalmaz jól kiegyensúlyozotnak tekinthető

In [ ]:
df_news_filtered["label_name"].hist()

### Decimális szám kódolású cimkék előállítása az egyes osztályokhoz

In [ ]:
label = []
for item in df_news_filtered["label_name"].values:
    if item == "Business":
        label.append(0)
    elif item == "SciTech":
        label.append(1)
    elif item == "Sports":
        label.append(2)
    elif item == "World":
        label.append(3)
    else:
        label.append(0)

df_news_filtered["label"] = label

df_news_filtered["label"].hist()
df_news_filtered.head()

### Az adatok train, validációs és teszt adathalmazra való bontása

In [ ]:
len_df_news_filtered = len(df_news_filtered)

len_news_train = int(round(len_df_news_filtered * 0.6))
len_news_val_test = (len_df_news_filtered - len_news_train) // 8
len_news_train = len_news_train // 4

len_df_news_filtered, len_news_train, len_news_val_test

In [ ]:
df_news_filtered_0 = df_news_filtered.loc[df_news_filtered['label'] == 0].copy()
df_news_filtered_1 = df_news_filtered.loc[df_news_filtered['label'] == 1].copy()
df_news_filtered_2 = df_news_filtered.loc[df_news_filtered['label'] == 2].copy()
df_news_filtered_3 = df_news_filtered.loc[df_news_filtered['label'] == 3].copy()

df_news_filtered_0_train = df_news_filtered_0[:len_news_train].copy()
df_news_filtered_1_train = df_news_filtered_1[:len_news_train].copy()
df_news_filtered_2_train = df_news_filtered_2[:len_news_train].copy()
df_news_filtered_3_train = df_news_filtered_3[:len_news_train].copy()

df_news_filtered_0_val = df_news_filtered_0[len_news_train:len_news_train+len_news_val_test].copy()
df_news_filtered_1_val = df_news_filtered_1[len_news_train:len_news_train+len_news_val_test].copy()
df_news_filtered_2_val = df_news_filtered_2[len_news_train:len_news_train+len_news_val_test].copy()
df_news_filtered_3_val = df_news_filtered_3[len_news_train:len_news_train+len_news_val_test].copy()

df_news_filtered_0_test = df_news_filtered_0[len_news_train+len_news_val_test:].copy()
df_news_filtered_1_test = df_news_filtered_1[len_news_train+len_news_val_test:].copy()
df_news_filtered_2_test = df_news_filtered_2[len_news_train+len_news_val_test:].copy()
df_news_filtered_3_test = df_news_filtered_3[len_news_train+len_news_val_test:].copy()

### Az előállított tanuló, validációs és teszt adathalmazok ellenőrzése

Az ellenőrzés során jól látható, hogy a nyers adathalmaz felbontása tanuló, validációs és teszt adathalmazon olyan módon történt hogy a létrjövő adathalmazokban az adatok osztályok szerinti egyeneletes eloszlása megmaradt.

In [ ]:
df_news_filtered_train = pd.concat([df_news_filtered_0_train, 
                                    df_news_filtered_1_train, 
                                    df_news_filtered_2_train,
                                    df_news_filtered_3_train])
df_news_filtered_train = df_news_filtered_train.sample(frac=1)
df_news_filtered_train.info()
df_news_filtered_train.hist()

In [ ]:
df_news_filtered_val = pd.concat([df_news_filtered_0_val, 
                                    df_news_filtered_1_val, 
                                    df_news_filtered_2_val,
                                    df_news_filtered_3_val])
df_news_filtered_val = df_news_filtered_val.sample(frac=1)
df_news_filtered_val.info()
df_news_filtered_val.hist()

In [ ]:
df_news_filtered_test = pd.concat([df_news_filtered_0_test, 
                                    df_news_filtered_1_test, 
                                    df_news_filtered_2_test,
                                    df_news_filtered_3_test])
df_news_filtered_test = df_news_filtered_test.sample(frac=1)
df_news_filtered_test.info()
df_news_filtered_test.hist()

### Elkészített és ellenőrzött adathalmazok kiírása

In [ ]:
news_train = "data/news_train.json"
df_news_filtered_train.to_json(news_train, orient="records")
df_readed_news_filtered_train = pd.read_json(news_train, orient='records')

news_val = "data/news_val.json"
df_news_filtered_val.to_json(news_val, orient="records")
df_readed_news_filtered_val = pd.read_json(news_val, orient='records')

news_test = "data/news_test.json"
df_news_filtered_test.to_json(news_test, orient="records")

## Tanuló, validációs és teszt adatok beolvasása

In [ ]:
df_train = pd.read_json("data/news_train.json", orient="records")
df_val = pd.read_json("data/news_val.json", orient="records")
df_test = pd.read_json("data/news_test.json", orient="records")

# Adatok megtisztítása

A jobb eredmény elérése érdekében az adatokból eltávolításra kerülnek a speciális html és egyéb karakterek és stopwordok. A szövegek minden karakter kisbetűre lesz alakítva továbbá egyszerű szavankénti tokenizálással fel lesz darabolva.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

STOPWORDS = stopwords.words("english")

def is_html_tag(word):
    w = word.replace("\n", "")
    return w.startswith("<") or w.startswith(">") or w.startswith("/") or w.strip()[:2] == "br"

def remove_html_tags(sentence):
    single_spaces = " ".join(sentence.split())

    return " ".join([token for token in single_spaces.split(" ") if not is_html_tag(token)])

def tokenize_sentence(sentence):
    return word_tokenize(sentence.lower())

def remove_stopwords(sentence):
    return " ".join([word for word in sentence.split(" ") if not word in STOPWORDS])

def remove_punctuation(tokenized_sentence):
    return [word for word in tokenized_sentence if word.isalpha()]

def preprocess(sentence):
    return remove_punctuation(tokenize_sentence(remove_stopwords(remove_html_tags(str(sentence)))))


In [ ]:
df_train["text"] = df_train["text"].apply(preprocess)
df_train.head()

## Szótár építése a tokenizált szövegből. 

Erre azért van szükség hogy megtudjuk határozni a szöveg szókészletét és ez alapján a gyakorisági táblázatott elő tudjuk állítani.

In [ ]:
def build_vocab(tokenized_input, vocab_size):
    d = dict()

    for tokens in tokenized_input:
        for token in tokens:
            # double check
            if token not in STOPWORDS and token.isalpha():
                d[token] = d.get(token, 0) + 1

    del d["br"]

    return {k for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)[:vocab_size]}

In [ ]:
VOCAB_SIZE = 10000
VOCAB = build_vocab(df_train["text"].values, VOCAB_SIZE) 
len(VOCAB)

## Tanuló cimkék előállítása és leképzése

In [ ]:
LABELS = df_train["label"].unique()
LABELS

In [ ]:
LABELS_TO_NAMES = {label: df_train[df_train["label"] == label].iloc[0]["label_name"] for label in LABELS}
LABELS_TO_NAMES

## Szavak gyakorsiágának meghatározása

In [ ]:
def get_frequencies_for_labels(df):
    dict_freqs = {label: {} for label in LABELS}

    for idx in range(df.shape[0]):
        tokens = df.iloc[idx, 0]
        label = df.iloc[idx, 1]
        
        for token in set(tokens):
            if token in VOCAB:
                dict_freqs[label][token] = dict_freqs[label].get(token, 0) + 1
            

    return dict_freqs

frequency_table = get_frequencies_for_labels(df_train)

# A money szavak gyakroisága az egyes osztályokban
'Business', frequency_table[0]["money"], 'SciTech', frequency_table[1]["money"], 'Sports', frequency_table[2]["money"], 'World', frequency_table[3]["money"]

## Tanuló jellemzők és cimkék elkészítése

Minden egyes tanuló mintához meghatározásra kerül, hogy a benne lévő szavak milyen gyakorisággal vesznek részt az egyes osztályokba. Ezáltal minden egyes szöveghez leképezhető egy elem négyesre. Ezekhez fogja az optimális súlyokat megkeresni a logisztukus model

In [ ]:
import numpy as np

def extract_features(frequency_table, tweet_tokens):
    label_frequencies = {label: 0 for label in LABELS}

    for t in tweet_tokens:
        for label in LABELS:
            label_frequencies[label] += frequency_table[label].get(t, 0)
    
    return pd.Series(label_frequencies)


In [ ]:
X_train_logistic = df_train["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
X_train_logistic

In [ ]:
y_train_logistic = df_train["label"].values
y_train_logistic

## Logisztikus model alkalmazása

### Validációs adathalmaz előkészítése az optimális modell megtalálásához

In [ ]:
X_val = df_val["text"].apply(preprocess)
X_val_logistic = df_val["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
y_val_logistic = df_val["label"].values
X_val_logistic

### Tanítás és validáció

In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

results = { "iter" : [], "Train accuracy" : [], "Validation accuracy" : []}
X_val = df_val["text"].apply(preprocess)

for max_iter in tqdm([2, 3, 4, 5, 10, 15]):
    clf = LogisticRegression(random_state=0, max_iter=max_iter, solver="newton-cg").fit(X_train_logistic, y_train_logistic)
    
    preds_train = clf.predict(X_train_logistic)
    preds_val = clf.predict(X_val_logistic)

    results["iter"].append(max_iter)
    results["Train accuracy"].append(accuracy_score(y_train_logistic, preds_train))
    results["Validation accuracy"].append(accuracy_score(y_val_logistic, preds_val))

results = pd.DataFrame(results)
results

In [ ]:
best_iter = results["iter"].values[np.argmax(results["Validation accuracy"].values, axis=0)]
best_iter

### Test adathalmaz előkészítése

In [ ]:
X_test = df_test["text"].apply(preprocess)
X_test_logistic = df_test["text"].apply(lambda tokens: extract_features(frequency_table, tokens))
y_test_logistic = df_test["label"].values
X_test_logistic

### A legjobb modell tesztelése

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=best_iter, solver="newton-cg").fit(X_train_logistic, y_train_logistic)

preds_test = clf.predict(X_test_logistic)

print("Test accuracy:", accuracy_score(y_test_logistic, preds_test))